In [13]:
from datasets import load_dataset
from transformers import TrainingArguments
import numpy as np

# Dataset with context, this might be necessary for model to pickup proper thing to respond to. In the repo there's also a dataset with all questions and answers but I doubt it'd be as useful
dataset = load_dataset("tylercross/platos_socrates")
dataset.shape

{'train': (3550, 3)}

In [96]:
from unittest.util import _MAX_LENGTH
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")

def tokenize_function(examples):
    tokenizer.pad_token = tokenizer.eos_token
    max_length = 2048
    # return tokenizer(examples["text"], padding="max_length", truncation=True)
    return tokenizer(examples["input"]+examples['output'], padding="max_length", return_tensors='pt', truncation=True, max_length=2048)

tokenized_dataset = dataset['train'].map(tokenize_function, batched=True, batch_size=1, drop_last_batch=True)
# Absloutely necessary otherwise no loss error
tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_datasets["input_ids"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [102]:
train = tokenized_dataset.select(range(100))

In [83]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")

In [85]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,


  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
#   /max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir='./',

  # Other arguments
  overwrite_output_dir=True, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="epoch",
  save_strategy="epoch",
  logging_strategy="epoch",
  logging_steps=1,

  
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False,

  # see here, avoid oom https://discuss.huggingface.co/t/cuda-out-of-memory-when-using-trainer-with-compute-metrics/2941/2
  eval_accumulation_steps=10
)



In [104]:
# No clue why but when I switch data to the Taylor Swift dataset, trainer gives  expected sequence of length 30 at dim 1 (got 16), when all other parts of the code are the same
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train
,
    eval_dataset=train,
    # compute_metrics=compute_metrics,
)

trainer.train()

 33%|███▎      | 25/75 [00:53<01:44,  2.09s/it]

{'loss': 2.2962, 'grad_norm': 11032348.0, 'learning_rate': 6.7567567567567575e-06, 'epoch': 1.0}





































































































                                               

 33%|███▎      | 25/75 [01:08<01:44,  2.09s/it]  



{'eval_loss': 0.5954952836036682, 'eval_runtime': 15.082, 'eval_samples_per_second': 6.63, 'eval_steps_per_second': 6.63, 'epoch': 1.0}


 67%|██████▋   | 50/75 [02:01<00:52,  2.09s/it]

...





































































































                                               

100%|██████████| 75/75 [03:25<00:00,  2.17s/it]  



{'eval_loss': 0.5668225288391113, 'eval_runtime': 15.0793, 'eval_samples_per_second': 6.632, 'eval_steps_per_second': 6.632, 'epoch': 3.0}



100%|██████████| 75/75 [03:26<00:00,  2.75s/it]

{'train_runtime': 206.0711, 'train_samples_per_second': 1.456, 'train_steps_per_second': 0.364, 'train_loss': 1.1494413630167644, 'epoch': 3.0}


TrainOutput(global_step=75, training_loss=1.1494413630167644, metrics={'train_runtime': 206.0711, 'train_samples_per_second': 1.456, 'train_steps_per_second': 0.364, 'train_loss': 1.1494413630167644, 'epoch': 3.0})

In [111]:
def generate_output(test_question, model, max_input_tokens=1000, max_output_tokens=100):
    tokenizer.pad_token = tokenizer.eos_token

    # Tokenize
    input_ids = tokenizer.encode(
          test_question,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens,
  
    )

    # Generate
    device = model.device
    generated_tokens_with_prompt = model.generate(input_ids=input_ids.to(device), max_length=max_output_tokens)

    # Decode
    generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

    # Strip the prompt
    generated_text_answer = generated_text_with_prompt[0][len(test_question):]
    print(generated_text_answer)
    return generated_text_answer

In [119]:
generate_output("""Who was the person""", model=model, max_output_tokens=100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 who was the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first


' who was the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first to be the first'

### Lora/Qlora using PEFT: [Good Tutorialx](https://www.youtube.com/watch?v=XpoKB3usmKc&t=1292s)

In [122]:
import peft
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

In [ ]:
peft_config =